In [2]:
# import torch
# from torch.autograd import Variable
# from warpctc_pytorch import CTCLoss
# ctc_loss = CTCLoss()

In [99]:
import librosa
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import os
import random
import numpy as np

In [101]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor
zeros = to

In [102]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [215]:
def pad_to_batch(batch):
    x,y = zip(*batch)
    max_x = max([s.size(0) for s in x])
    x_p=[]
    for i in range(len(batch)):
        if x[i].size(0)<max_x:
            pad = torch.cat([x[i],Variable(torch.zeros(max_x-x[i].size(0),20))]).unsqueeze(0)
            if USE_CUDA:
                pad=pad.cuda()
            x_p.append(pad)
        else:
            x_p.append(x[i].unsqueeze(0))
    return torch.cat(x_p),torch.cat(y)

In [72]:
BASE_URL = '../../다운로드/google_command_wav/'

In [90]:
data=[]

In [91]:
for yes in os.listdir(BASE_URL+'yes/'):
    if yes.endswith('.wav'):
        y, sr = librosa.load(BASE_URL+'yes/'+yes,sr=16000)
        feature = librosa.feature.mfcc(y,sr)
        data.append([feature,1])
        
for no in os.listdir(BASE_URL+'no/'):
    if no.endswith('.wav'):
        y, sr = librosa.load(BASE_URL+'no/'+no,sr=16000)
        feature = librosa.feature.mfcc(y,sr)
        data.append([feature,0])

In [208]:
random.shuffle(data)
train = data[:int(len(data)*0.7)]
test = data[int(len(data)*0.7):]

In [209]:
train_X,train_y = list(zip(*train))
test_X,test_y = list(zip(*test))

In [210]:
train_X = [Variable(FloatTensor(x.T)) for x in train_X]
train_y = [Variable(LongTensor([y])).unsqueeze(0) for y in train_y]
train_data = list(zip(train_X,train_y))

In [211]:
test_X = [Variable(FloatTensor(x.T)) for x in test_X]
test_data = list(zip(test_X,test_y))

In [191]:
class YesNoClassifier(nn.Module): 
    def __init__(self,vector_size,hidden_size):

        super(YesNoClassifier, self).__init__()
        self.vector_size = vector_size
        self.hidden_size = hidden_size
        self.rnn = nn.GRU(vector_size,hidden_size,1,batch_first=True)
        self.linear = nn.Linear(hidden_size,2)
        
        self.init_rnn() # IRNN
            
    def init_rnn(self):
        self.rnn.weight_hh_l0 = nn.init.eye(self.rnn.weight_hh_l0)
        self.rnn.weight_ih_l0 = nn.init.eye(self.rnn.weight_ih_l0)
        self.rnn.bias_hh_l0.data.fill_(0)
        self.rnn.bias_ih_l0.data.fill_(0)
        
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(1,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
        
    def forward(self, inputs): 
        hidden = self.init_hidden(inputs)
        out,hidden = self.rnn(inputs,hidden)
        return F.log_softmax(self.linear(hidden.squeeze(0)))
#         return F.log_softmax(self.linear(out.contiguous().view(out.size(0)*out.size(1),-1)))

In [194]:
model = YesNoClassifier(20,50)
if USE_CUDA:
    model = model.cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [195]:
for step in range(2):
    losses=[]
    for i,batch in enumerate(getBatch(5,train_data)):
        inputs,targets = pad_to_batch(batch)
        model.zero_grad()
        preds = model(inputs)

        loss = loss_function(preds,targets.view(-1))
        losses.append(loss.data.cpu().numpy()[0] if USE_CUDA else loss.data.numpy()[0] )
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5) # gradient clipping
        optimizer.step()
        
        
        if i % 100==0:
            print("[%d/%d] mean_loss : %0.2f" %(step,5,np.mean(losses)))
            losses=[]

[0/5] mean_loss : 0.68
[0/5] mean_loss : 0.68
[0/5] mean_loss : 0.28
[0/5] mean_loss : 0.24
[0/5] mean_loss : 0.23
[0/5] mean_loss : 0.25
[0/5] mean_loss : 0.18
[1/5] mean_loss : 0.07
[1/5] mean_loss : 0.14
[1/5] mean_loss : 0.20
[1/5] mean_loss : 0.15
[1/5] mean_loss : 0.16
[1/5] mean_loss : 0.19
[1/5] mean_loss : 0.20


In [216]:
accuracy=0

In [217]:
for test in test_data:
    pred = model(test[0].unsqueeze(0)).max(1)[1]
    pred = pred.data.cpu().tolist()[0] if USE_CUDA else pred.data.tolist()[0]
    if pred == test[1]:
        accuracy+=1

print(accuracy/len(test_data)*100)

95.37166900420758
